In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    break
    if 'art-seg-ready' in dirname:
        continue
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install evaluate

In [ ]:
import zipfile
import glob
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import random


from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch
from torchvision import datasets,models
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import v2
from torchvision.transforms import ColorJitter
from transformers import SegformerImageProcessor
import evaluate
from torch import nn
import torch.nn.functional as F
import wandb
import scipy


np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [ ]:
run_name = 'SegFormer-full-cont-jitter'
wandb.init(project='aimlproject', name=run_name,save_code=True, config={
            "epochs": 2,
           "batch_size": 32,
              "lr": 5e-5,
          "checkpoint": "nvidia/segformer-b0-finetuned-ade-512-512",
           "model_name":"SegFormer", "early_stopping":False})  # mode='disabled',
config = wandb.config

In [ ]:
wandb.config


In [ ]:
id2label = {
            0: 'background',
            1: 'bird',
            2: 'boat',
            3: 'bottle',
            4: 'cat',
            5: 'chair',
            6: 'cow',
            7: 'dog',
            8: 'horse',
            9: 'person',
            10: 'pottedplant',
            11: 'sheep',
}

label2id = {v: k for k, v in id2label.items()}
model_checkpoint = config.checkpoint


In [ ]:
EXPR_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_expressionism2/kaggle/working/pascal_sbd_styled_expressionism'
IMPR_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_impressionism2/kaggle/working/pascal_sbd_styled_impressionism'
PIMP_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_post_impressionism2/kaggle/working/pascal_sbd_styled_post_impressionism'
REAL_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_realism2/kaggle/working/pascal_sbd_styled_realism'
PATHS = [ EXPR_PATH, IMPR_PATH, PIMP_PATH, REAL_PATH ]
PATHS_IM = [ os.path.join(path, 'images') for path in PATHS ]
PATHS_LAB = [ os.path.join(path, 'labels') for path in PATHS ]

In [ ]:
PATHS

In [ ]:
#print(os.listdir('/kaggle/input/art-seg-ready/pascal_sbd_styled_realism1/pascal_sbd_styled_realism1/kaggle/working/pascal_sbd_styled_realism/images'))
def plot_random_images(allstyles=True,path=''):
    selected_images = []
    if allstyles:
    #image_files = [f for f in os.listdir(basepath + '/dataset'+config.dataset+trainfolder) if f.startswith(startchar)]
        shuffled_paths = PATHS_IM
        random.shuffle(shuffled_paths)
        
        for i in range(len(PATHS)-1):
            path = shuffled_paths[i]
            image_files = os.listdir(path)
            image_files = random.sample(image_files, 2) 
            image_files = [ os.path.join(path, image) for image in image_files]
            selected_images += image_files
            
            
        path = shuffled_paths[3]

        image_files = os.listdir(path)
        image_files = random.sample(image_files,3)
        image_files = [ os.path.join(path, image) for image in image_files]
        selected_images += image_files
             
    else:
        if path == '':
            print('empty path, useless')
            return 
        image_files = os.listdir(path)
        image_files = random.sample(image_files, 9)
        image_files = [ os.path.join(path, image) for image in image_files]
        selected_images += image_files

    #selected_images = random.sample(image_files, 9)

    rows = 3
    cols = 3
    fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

    # Plot each selected image
    for i, image_name in enumerate(selected_images):
        #img_path = os.path.join(basepath + '/dataset'+config.dataset+trainfolder, image_name)
        img = mpimg.imread(image_name)
        ax = axes[i // cols, i % cols]
        ax.imshow(img)
        ax.axis('off')

    # Adjust layout
    plt.tight_layout()
    plt.show()

plot_random_images()


In [ ]:
train_list = []
for i in range(4):
    style_list = os.listdir(PATHS_LAB[i])
    style_list = [ os.path.join(PATHS_LAB[i], image) for image in style_list]
    train_list += style_list
    
    
#train_list,test_list = train_test_split(train_list, test_size=0.2, random_state=0)
train_list,val_list = train_test_split(train_list, test_size=0.2, random_state=0)

#print(train_list[1], train_list[-1])
example_conv = train_list[2020]
example_conv = example_conv[:-4] + '.jpg'
#example_conv = os.path.join()
example_conv = example_conv.replace("/labels/", "/images/")
print(example_conv)

In [ ]:
train_list[0]
val_list[0]

In [ ]:
class PaintingSegDataset(Dataset):
    def __init__(self, file_list, transform=None,use_gpu=torch.cuda.is_available()): #df(1st)
        self.file_list = file_list
        self.transform = transform
        self.filelength = len(file_list)
        self.jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 
        self.use_gpu = use_gpu
        self.id_to_trainid = {
            0: 0,
            3: 1,
            4: 2,
            5: 3,
            8: 4,
            9: 5,
            10: 6,
            12: 7,
            13: 8,
            15: 9,
            16: 10,
            17: 11,
        }
        self.debug = True


    def __len__(self):
        return self.filelength

    def __getitem__(self, idx):
        label_path = self.file_list[idx]
        
        img_path = label_path[:-4] + '.jpg'
        img_path = img_path.replace("/labels/", "/images/")
        #print(img_path)
        img_orig = Image.open(img_path).convert('RGB')
        
        if label_path[-3:] == 'png':
            label = np.array(Image.open(label_path), dtype=np.float32)
        else:  # .mat
            label = scipy.io.loadmat(label_path)["GTcls"][0]['Segmentation'][0]
            
        #?name =
        #print(label)
        label_copy = 255 * np.ones(label.shape, dtype=np.int8)
        for k, v in self.id_to_trainid.items():
            label_copy[label == k] = v
        label = Image.fromarray(label_copy)
        #label = label.convert('L')
        #transform = v2.Compose([v2.Resize((512,512))])
        #img_orig = transform(img_orig)
        #print(tensor.unique())  
        img_jitter = self.jitter(img_orig)
        inputs = self.transform(images=img_jitter, segmentation_maps=label, return_tensors="pt", do_reduce_labels=False)
        #print(inputs)
        img = inputs["pixel_values"].squeeze()  # Remove batch dimension
        label = inputs["labels"].squeeze()
        #print(img.shape)
        #print(label.shape)
        if self.use_gpu:
            img = img.cuda()
            label = label.cuda()
       
        return img, label

In [ ]:

image_processor = SegformerImageProcessor.from_pretrained(model_checkpoint)

####test_dataset = PaintingSegDataset(test_list,transform=image_processor) #df
val_dataset = PaintingSegDataset(val_list,transform=image_processor) #df
train_dataset = PaintingSegDataset(train_list,transform=image_processor)


In [ ]:
vizualize=False

def color_palette():
    """Color palette that maps each class to RGB values.
    
    This one is actually taken from ADE20k.
    """
    return [[120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]]
if vizualize:
    palette = color_palette()

    v_index = 101
    segmentation_map = train_dataset[v_index][1]
    image = train_dataset[v_index][2]
    print(segmentation_map)

    color_segmentation_map = np.zeros((segmentation_map.shape[0], segmentation_map.shape[1], 3), dtype=np.uint8) # height, width, 3
    for label, color in enumerate(palette):
        color_segmentation_map[segmentation_map - 1 == label, :] = color
    #print(ground_truth_color_seg.shape)
    # Convert to BGR
    ground_truth_color_seg = color_segmentation_map[..., ::-1]
    img = np.array(image)
    print(img.shape)
    #img = np.transpose(img)
    print(img.shape)
    img = img * 0.5 + ground_truth_color_seg * 0.5
    img = img.astype(np.uint8)

    plt.figure(figsize=(15, 10))
    plt.imshow(img)
    plt.show()

In [ ]:
batch_size = config.batch_size 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
dataiter = iter(train_loader)
a,b = next(dataiter)
print(a.shape)
print(b.shape)

In [ ]:
from transformers import SegformerForSemanticSegmentation
import requests

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=12,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,  # Will ensure the segmentation specific components are reinitialized.
)



model.load_state_dict(torch.load('/kaggle/input/segformer-weights/pytorch/default/3/SegFormerSegFormer-full3.pth'))

#model.config

In [ ]:
print(model.state_dict()['decode_head.linear_c.0.proj.weight'])
#print(model1.state_dict()['decode_head.linear_c.0.proj.weight'])



In [ ]:
#print(model["segformer.encoder.patch_embeddings.0.proj"].weight)
# Freeze all the layers first
for i, (name, param) in enumerate(model.named_parameters()):
    if not "decode_head" in name:
        param.requires_grad = False
        
    #decoder_layers_to_freeze = ["0","1","2"]
    #if any(el in name for el in decoder_layers_to_freeze):
    #    param.requires_grad = False


for name, param in model.named_parameters():
    break
    print(f"{name}: {param.requires_grad}")

In [ ]:
from transformers import get_scheduler
from tqdm.auto import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr) #config.lr
model.to(device)
num_epochs = config.epochs
num_training_steps = num_epochs * len(train_loader)
num_val_steps = num_epochs * len(valid_loader)


lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
metric = evaluate.load("mean_iou")

def compute_metrics(logits,labels,per_category=False):
  with torch.no_grad():
    #logits, labels = eval_pred
    #logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    # currently using _compute instead of compute
    # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=12,
            ignore_index=255,
            reduce_labels=False,
        )
    if per_category:
        # add per category metrics as individual key-value pairs
        per_category_miou = metrics.pop("per_category_miou").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        metrics.update({f"miou_{id2label[i]}": v for i, v in enumerate(per_category_miou)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_val = tqdm(range(num_val_steps))
#wandb.watch(model, log_freq=100)


model.train()
cnt = 0
for epoch in range(num_epochs):
    training_loss_epoch = 0
    training_miou_epoch = 0
    
    for batch_idx, (train_features_batch, train_labels_batch) in enumerate(train_loader):
        optimizer.zero_grad()

        outputs = model(train_features_batch,train_labels_batch)
        
        loss = outputs.loss # NllLoss2DBackward0
            
        metrics = compute_metrics(outputs['logits'], train_labels_batch)
        training_loss_epoch += loss.item()
        training_miou_epoch += metrics['mean_iou']
            
        loss.backward()
        optimizer.step() 
        lr_scheduler.step()
        progress_bar_train.update(1)
        
        
    validation_loss_epoch = 0
    validation_miou_epoch = 0
    
    
    for val_features_batch, val_labels_batch in valid_loader:

        with torch.no_grad():       
        
            outputs = model(val_features_batch, val_labels_batch)

            loss = outputs.loss # NllLoss2DBackward0
            
            metrics = compute_metrics(outputs['logits'], val_labels_batch)
            
            validation_loss_epoch += loss.item()
            validation_miou_epoch += metrics['mean_iou']
            
            progress_bar_val.update(1)
            
    print("Saving model weights")
    torch.save(model.state_dict(), 'SegFormer'+ str(run_name) + str(epoch) + '.pth') 
        
    print({'training_loss_epoch': training_loss_epoch/len(train_loader), 'validation_loss_epoch': validation_loss_epoch/len(valid_loader)})
    print({'training_miou_epoch': training_miou_epoch/len(train_loader), 'validation_miou_epoch': validation_miou_epoch/len(valid_loader)})
    wandb.log({'training_loss_epoch': training_loss_epoch/len(train_loader), 'validation_loss_epoch': validation_loss_epoch/len(valid_loader),
               'training_miou_epoch': training_miou_epoch/len(train_loader), 'validation_miou_epoch': validation_miou_epoch/len(valid_loader)})

In [ ]:
import gc
#nekaj   
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
wandb.finish()